In [36]:
function eye(n::Int)::Matrix{Float64}
    return Matrix{Float64}(I,n,n)
end

eye (generic function with 1 method)

In [37]:
using LinearAlgebra
A = [1 4 1;
     4 3 2;
     1 2 5]

3×3 Matrix{Int64}:
 1  4  1
 4  3  2
 1  2  5

In [38]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -2.1529137088817887
  3.321327095067036
  7.831586613814752
vectors:
3×3 Matrix{Float64}:
  0.77535   -0.428782  -0.463658
 -0.627948  -0.445339  -0.638243
  0.067182   0.786014  -0.614547

In [39]:
# obliczanie równan wysokiego rzedu, wyznaczanie współczynnika uwarunkowania macierzy (jak niewielka zmiana macierzy A zmienia wynik Ax=b),
# poszukiwanie promienia spektralnego - największa wartość wlasna macierzy co do modułu

function calculate_eigen_value(x::Vector, A::Matrix)
    return x'*A*x
end

function power_max_eigen(A::Matrix, acc=1e-10::Float64)::Vector
    n = size(A, 1)
    x = [1:n;];
    max_eigen_value = 1.0;
    
    while true
        prev_max_val = calculate_eigen_value(x, A);
        y = A*x;
        x = y/norm(y);
        max_eigen_value = calculate_eigen_value(x, A) #uproszczony wzor bez mianownika poniewaz dlugosc wetkora x jest = 1 ze wzgledu na wprowadzoną nrmalizację w pętli

        curr_acc = abs(max_eigen_value/prev_max_val);
        if curr_acc < acc+1 && curr_acc > 1
            break
        end
    end
    
    return [max_eigen_value, x]
end


power_max_eigen (generic function with 2 methods)

In [52]:
eigen_julia = eigmax(A);
eigen_custom, x = power_max_eigen(A);

println("Custom max eigen:")
println(eigen_custom)
println("\n")
println("Julia max eigen:")
println(eigen_julia)

-2.152913708881789

In [40]:
# deflacja
max_eigen, x = power_max_eigen(A);
display(x)
display(max_eigen)

An = A - max_eigen*x*x';
sec_max_eigen, x = power_max_eigen(An)
display(x)
display(sec_max_eigen)

Ann = An - sec_max_eigen*x*x';
third_max_eigen, x = power_max_eigen(Ann)
display(x)
display(third_max_eigen)


3-element Vector{Float64}:
 0.46365587190028185
 0.6382407630306943
 0.6145502102012385

7.831586613736695

3-element Vector{Float64}:
 -0.42878257244942775
 -0.4453490568340997
  0.7860087296847467

3.321327095098886

3-element Vector{Float64}:
 -0.7753464274567067
  0.6279519421093321
 -0.06718836083161735

-2.1529137091165995

In [41]:
function power_min_eigen(A::Matrix, acc=1e-10::Float64)::Vector
    n = size(A, 1)
    x = [1:n;];
    min_eigen_value = 1.0;
    
    while true
        prev_min_val = calculate_eigen_value(x, A);
        y = A\x;
        x = y/norm(y);
        min_eigen_value = calculate_eigen_value(x, A) #uproszczony wzor bez mianownika poniewaz dlugosc wetkora x jest = 1 ze wzgledu na wprowadzoną nrmalizację w pętli

        curr_acc = abs(min_eigen_value/prev_min_val);
        if curr_acc <= acc+1 && curr_acc >= 1
            break
        end
    end
    
    return [min_eigen_value, x]
end

power_min_eigen (generic function with 2 methods)

In [54]:
min_eigen, x = power_min_eigen(A);
eigen_julia = eigmin(A);

println("Custom min eigen:")
println(min_eigen)
println("\n")
println("Julia min eigen:")
println(eigen_julia)

Custom min eigen:
-2.1529137088142187


Julia min eigen:
-2.152913708881789


In [45]:
# współczynnik uwarunkowania macierzy:
function get_cond(A::Matrix)::Float64
    n = size(A, 1)
    A_A = A'*A;
    max_eigen, x = power_max_eigen(A_A);
    A_A_min = A_A - max_eigen*eye(n);
    min_eigen_dash, x = power_max_eigen(A_A_min);
    min_eigen = min_eigen_dash + max_eigen;
    cond_mx = sqrt(max_eigen/min_eigen)
    return cond_mx
end

cond_custom = get_cond(A)

const_julia = cond(A)

println("Custom cond:")
println(cond_custom)
println("\n")
println("Julia cond:")
println(const_julia)


Custom cond:
3.637668600207373


Julia cond:
3.637668607666786
